In [ ]:
# instsall tar2tf module
! pip3 install ../

In [1]:
# list available ais buckets
!ais ls ais://

AIS Buckets (1)
  tar-bucket


In [2]:
# list contents of your bucket
!ais ls ais://tar-bucket

NAME		 SIZE		 VERSION	 
train-0.tar	17.48MiB	1		
train-1.tar	17.48MiB	1		
train-10.tar	17.48MiB	1		
train-11.tar	17.48MiB	1		
train-12.tar	17.48MiB	1		
train-13.tar	17.48MiB	1		
train-14.tar	17.48MiB	1		
train-15.tar	17.48MiB	1		
train-16.tar	17.48MiB	1		
train-17.tar	17.48MiB	1		
train-18.tar	17.48MiB	1		
train-19.tar	17.48MiB	1		
train-2.tar	17.48MiB	1		
train-20.tar	17.48MiB	1		
train-3.tar	17.48MiB	1		
train-4.tar	17.48MiB	1		
train-5.tar	17.48MiB	1		
train-6.tar	17.48MiB	1		
train-7.tar	17.48MiB	1		
train-8.tar	17.48MiB	1		
train-9.tar	17.48MiB	1		


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tar2tf import AisDataset
from tar2tf.ops import Select, Decode, Convert, Resize, Rotate, Func

EPOCHS = 5
BATCH_SIZE = 20

# ADJUST AisDataset PARAMETERS BELOW

BUCKET_NAME = "tar-bucket"
PROXY_URL = "http://localhost:8080"

INPUT_SHAPE = (224, 224, 3)

In [ ]:
# Create AisDataset.
# tar records will be transformed according [Decode("jpg"), Rotate("jpg"), Resize("jpg", (224, 224)) operations,
# meaning that bytes under "jpg" in tar-record will be decoded as an image, Rotated by random angle and then Resized to (224, 224)
# Datapoints will be constructed from "jpg" and "cls" tar records entries
conversions = [Decode("jpg"), Rotate("jpg"), Resize("jpg", (224, 224))]
selections = ["jpg", "cls"]
ais = AisDataset(BUCKET_NAME, PROXY_URL, conversions, selections)

# Prepare your bucket first with Gavin's tars (gsutil ls gs://lpr-gtc2020)
# Remote conversions and selections execution by default
# Datasets will be prepared from "train-{0..5}.tar" files, records of each tar file will be shuffled within a scope of a tar file
# Prefetches BATCH_SIZE, limits dataset first BATCH_SIZE * 5 elements, caches them, repeats forever and creates batches from infinite dataset
train_dataset = ais.load("train-{0..5}.tar", num_workers=5, shuffle_tar=True, output_shapes=(tf.TensorShape(INPUT_SHAPE), tf.TensorShape([None]))
).take(BATCH_SIZE * 5).cache().repeat().batch(BATCH_SIZE)
test_dataset = ais.load("train-{6..10}.tar", num_workers=5, output_shapes=(tf.TensorShape(INPUT_SHAPE), tf.TensorShape([None]))
).take(BATCH_SIZE).cache().batch(BATCH_SIZE)

In [ ]:
# TRAINING PART BELOW
inputs = keras.Input(shape=(
    224,
    224,
    3,
), name="images")
x = layers.Flatten()(inputs)
x = layers.Dense(64, activation="relu", name="dense_1")(x)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=keras.optimizers.Adam(1e-4), loss=keras.losses.mean_squared_error, metrics=["acc"])
model.summary()

model.fit(train_dataset, epochs=EPOCHS, steps_per_epoch=BATCH_SIZE)
result = model.evaluate(test_dataset)
print(dict(zip(model.metrics_names, result)))
